In [2]:
%pip install tqdm Pillow ipywidgets jupyter ipykernel

  Using cached nbformat-5.10.4-py3-none-any.whl.metadata (3.6 kB)
  Using cached jsonschema-4.23.0-py3-none-any.whl.metadata (7.9 kB)
  Using cached fastjsonschema-2.21.1-py3-none-any.whl.metadata (2.2 kB)
  Using cached jsonschema_specifications-2024.10.1-py3-none-any.whl.metadata (3.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.4/214.4 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 55.1 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 113.8 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.5/258.5 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 121.7 MB/s eta 0:00:0000:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.4/163.4 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/7

In [ ]:
import os
import pathlib
from PIL import Image
import multiprocessing
from functools import partial
import concurrent.futures
from tqdm.notebook import tqdm  # Using notebook version of tqdm

# Set these variables to your desired paths and size
parent_dir = '/Volumes/External SSD/rxrx3/'
output_dir = '/Volumes/External SSD/rxrx3_resized/'
target_size = (512, 512)  # Change this if you want a different size

# Function to resize a single image
def resize_image(src_path, dst_path):
    try:
        # Create destination directory if needed
        os.makedirs(os.path.dirname(dst_path), exist_ok=True)

        # Open, resize and save the image
        with Image.open(src_path) as img:
            # Resize with antialiasing
            resized_img = img.resize(target_size, Image.LANCZOS)
            # Save the resized image
            resized_img.save(dst_path)
        return True
    except Exception as e:
        return (src_path, str(e))

# Get all image paths from a directory
def get_image_paths(source_dir, dest_dir):
    image_paths = []

    # Get all directories first
    all_dirs = list(os.walk(source_dir))

    # Display progress for directory scanning
    for root, _, files in tqdm(all_dirs, desc="Scanning directories"):
        rel_path = os.path.relpath(root, source_dir)

        for file in files:
            if file.lower().endswith((".png", ".jpg", ".jpeg", ".tif", ".tiff")):
                src_path = os.path.join(root, file)

                # Construct destination path
                if rel_path == ".":
                    dst_path = os.path.join(dest_dir, file)
                else:
                    dst_path = os.path.join(dest_dir, rel_path, file)

                image_paths.append((src_path, dst_path))

    return image_paths

# Process a single directory
def process_directory(input_dir, output_base_dir):
    """Process a single directory of images"""
    
    # Get directory name and create output dir name with _512 suffix
    dir_name = os.path.basename(input_dir)
    output_dir = os.path.join(output_base_dir, f"{dir_name}_512")
    
    # Create output directory
    os.makedirs(output_dir, exist_ok=True)
    
    print(f"\nProcessing directory: {dir_name}")
    print(f"Output directory: {output_dir}")
    
    # Get all image paths
    image_paths = get_image_paths(input_dir, output_dir)
    print(f"Found {len(image_paths)} images to resize")
    
    if not image_paths:
        print(f"No images found in {input_dir}. Skipping...")
        return (0, 0)
    
    # Get number of CPU cores
    num_cores = multiprocessing.cpu_count()
    # Use 80% of available cores to avoid overloading the system
    num_workers = max(1, int(num_cores * 0.8))
    print(f"Using {num_workers} workers")
    
    # Process images in parallel
    results = []
    failures = []

    # Process in batches to show progress
    batch_size = 100  # Adjust based on memory constraints
    total_batches = (len(image_paths) + batch_size - 1) // batch_size

    for i in range(0, len(image_paths), batch_size):
        batch = image_paths[i:i + batch_size]
        batch_num = i // batch_size + 1
        print(f"Processing batch {batch_num}/{total_batches}")

        # Process current batch
        with concurrent.futures.ThreadPoolExecutor(max_workers=num_workers) as executor:
            # Map function directly to all items in batch
            batch_src_paths, batch_dst_paths = zip(*batch)
            for idx, result in enumerate(
                tqdm(
                    executor.map(resize_image, batch_src_paths, batch_dst_paths),
                    total=len(batch),
                    desc=f"Resizing images ({dir_name})"
                )
            ):
                if result is True:
                    results.append(result)
                else:
                    failures.append(result)
    
    print(f"\nResizing complete for {dir_name}!")
    print(f"Successfully resized: {len(results)}/{len(image_paths)} images")
    
    if failures:
        print(f"Failed to resize: {len(failures)} images")
        print("First few errors:")
        for src_path, error in failures[:5]:
            print(f"- {src_path}: {error}")

        if len(failures) > 5:
            print(f"... and {len(failures) - 5} more errors")
            
    return (len(results), len(failures))

# Main execution code
# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Get subdirectories
subdirs = [os.path.join(parent_dir, d) for d in os.listdir(parent_dir) 
          if os.path.isdir(os.path.join(parent_dir, d))]

if not subdirs:
    print(f"No subdirectories found in {parent_dir}")
else:
    print(f"Found {len(subdirs)} subdirectories to process:")
    for d in subdirs:
        print(f"- {os.path.basename(d)}")
    print("-" * 50)
    
    # Process each subdirectory
    total_success = 0
    total_failures = 0
    
    for subdir in subdirs:
        success, failures = process_directory(subdir, output_dir)
        total_success += success
        total_failures += failures
    
    # Final statistics
    print("\n" + "="*50)
    print("OVERALL STATISTICS:")
    print(f"Processed {len(subdirs)} directories")
    print(f"Successfully resized: {total_success} images")
    print(f"Failed: {total_failures} images")
    print("="*50)

Found 2 subdirectories to process:
- rxrx3
- rxrx3_temp
--------------------------------------------------

Processing directory: rxrx3
Output directory: /Volumes/External SSD/rxrx3_resized/rxrx3_512


Scanning directories:   0%|          | 0/6 [00:00<?, ?it/s]

Found 30510 images to resize
Using 12 workers
Processing batch 1/306


Resizing images (rxrx3):   0%|          | 0/100 [00:00<?, ?it/s]

Processing batch 2/306


Resizing images (rxrx3):   0%|          | 0/100 [00:00<?, ?it/s]

Processing batch 3/306


Resizing images (rxrx3):   0%|          | 0/100 [00:00<?, ?it/s]

Processing batch 4/306


Resizing images (rxrx3):   0%|          | 0/100 [00:00<?, ?it/s]